In [1]:
import matplotlib
matplotlib.use('agg')

In [2]:
%matplotlib inline

In [3]:
import phenom
import os
import scipy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
formula = """1 + C(mMAcid, Sum, levels=[5, 10, 20, 0]) 
               + C(pH, Sum, levels=[6.5, 6.0, 5.5, 5, 7]) 
               + C(mMAcid, Sum, levels=[5, 10, 20, 0]):C(pH, Sum, levels=[6.5, 6.0, 5.5, 5, 7])"""

# individual

In [ ]:
for acid in ["benzoate", "citric", "malic"]:
    ds = phenom.dataset.DataSet.fromDirectory("../data/pseudomonas/PA01-{}/".format(acid))
    meta = ds.meta
    meta["mMAcid"] = meta["mM-acid"]

    print("*"*40)
    print(acid)

    for plate in meta.plate.unique():
        m = meta.copy()
        d = ds.data.copy()

        select = m.plate == plate
        m = m.loc[select,:]
        d = d.loc[:, select]

        mnull = phenom.design.Formula(m,formula)
        hierarchy = phenom.design.Formula(m, "1") + phenom.design.Formula(m, "C(plate) + 0")
        mbatch = mnull * hierarchy

        phen = phenom.phenotype.Phenotype(d, mnull, model="phenom_deriv.stan")
        samples = phen.samples()
        print(name)
        print(samples)
        phen.save("../samples/pseudomonas/{}/individual/{}".format(acid, plate))
        print()

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_7226d8af40a102760f76e1d6f960daef NOW.


****************************************
benzoate


# full

In [ ]:
for acid in ["benzoate", "citric", "malic"]:
    ds = phenom.dataset.DataSet.fromDirectory("../data/pseudomonas/PA01-{}/".format(acid))
    meta = ds.meta
    meta["mMAcid"] = meta["mM-acid"]

    mnull = phenom.design.Formula(
        ds.meta,
        formula)
    hierarchy = phenom.design.Formula(ds.meta, "1") + phenom.design.Formula(ds.meta, "C(plate) + 0")
    mbatch = mnull * hierarchy

    null_phen = phenom.phenotype.Phenotype(ds.data, mnull, model="phenom_deriv.stan")
    batch_phen = phenom.phenotype.Phenotype(ds.data, mbatch, model="phenom_deriv.stan",
                                        lengthscale_priors=[[2.5, 0.45]] * 4 + [[100, 20]] * 4, 
                                        alpha_priors= [[10, 10]] * 4 + [[7, 10]] * 4,
                                        minExpectedCross=.01, maxExpectedCross=10,
                                        sigma_prior=[.02, 20])

    full_phen = phenom.phenotype.Phenotype(ds.data, 
                                        mbatch, model="phenom_marginal.stan", 
                                        lengthscale_priors=[[2.5, 0.45]] * 4 + [[100, 20]] * 4, 
                                        alpha_priors= [[10, 10]] * 4 + [[7, 10]] * 4,
                                        minExpectedCross=.01, maxExpectedCross=10,
                                        sigma_prior=[.02, 20],
                                        marginal_lengthscale_prior=[4, 1],
                                        marginal_alpha_prior=[2, 20]
                                        )

    print("*"*40)
    print(acid)

    for name, phen in [("null", null_phen), ("batch", batch_phen), ("full", full_phen)]:
        samples = phen.samples()
        print(name)
        print(samples)
        phen.save("../samples/pseudomonas/{}/{}".format(acid, name))
        print()

    print()

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_7226d8af40a102760f76e1d6f960daef NOW.


****************************************
benzoate
